#Enterprise Fleet Analytics Pipeline: Focuses on the business outcome (analytics) and the domain (fleet/logistics).

![logistics](logistics_project.png)

Download the data from the below gdrive and upload into the catalog
https://drive.google.com/drive/folders/1J3AVJIPLP7CzT15yJIpSiWXshu1iLXKn?usp=drive_link

In [0]:
%sql CREATE VOLUME IF NOT EXISTS workspace.default.logistics_vol;

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/default/logistics_vol/logistics_data")

##**1. Data Munging** -

####1. Visibily/Manually opening the file and capture couple of data patterns (Manual Exploratory Data Analysis) <br>

1. Duplicate rows observed
2. Additional columns observed
3. Values are not present in the given columns
4. file contains header and no footer


####2. Programatically try to find couple of data patterns applying below EDA (File: logistics_source1)
1. Apply inferSchema and toDF to create a DF and analyse the actual data.
2. Analyse the schema, datatypes, columns etc.,
3. Analyse the duplicate records count and summary of the dataframe.

In [0]:
# 1.Apply inferSchema and toDF to create a DF and analyse the actual data.
read_df = spark.read.csv("/Volumes/workspace/default/logistics_vol/logistics_data/logistics_source1",inferSchema = True,header=True)
display(read_df)

# 2.Analyse the schema, datatypes, columns etc.,
read_df.printSchema()
display(read_df.schema)
display(read_df.columns)
display(read_df.dtypes)

# 3.Analyse the duplicate records count and summary of the dataframe.
print("Total count of dataset",read_df.count())
print("de-duplicated record (all columns) count",read_df.distinct().count())        # distinct will remove entire columns duplicates and give the count of the records
print("de-duplicated record (all columns) count",read_df.dropDuplicates().count())  # dropDuplicates also will do the same, but we can work on selected columns
print("de-duplicated record (selected columns) count",read_df.dropDuplicates(['shipment_id']).count())

display(read_df.summary())


###a. Passive Data Munging -  (File: logistics_source1  and logistics_source2)
Without modifying the data, identify:<br>
Shipment IDs that appear in both master_v1 and master_v2<br>
Records where:<br>
1. shipment_id is non-numeric
2. age is not an integer<br>

Count rows having:<br>
3. fewer columns than expected <br>
4. more columns than expected

In [0]:
read_raw_df = spark.read.csv(path=["/Volumes/workspace/default/logistics_vol/logistics_data/logistics_source1","/Volumes/workspace/default/logistics_vol/logistics_data/logistics_source2"],header=True,inferSchema = True)
display(read_raw_df)

# 1. Shipment_id is non-numeric
from pyspark.sql.functions import col
#invalid_id_df = read_raw_df.filter(~col("shipment_id").rlike("^[0-9]+$").show()) # using Pyspark function
invalid_id_df = read_raw_df.where("shipment_id rlike '[a-zA-Z]'").show()   # using Sql function

# Optional - If we need to filter the string values with Null values

invalid_id_df = read_raw_df.filter((~col("shipment_id").rlike("^[0-9]+$"))| col("shipment_id").isNull()).show()  # using Pyspark function
invalid_id_df = read_raw_df.where("shipment_id rlike '[a-zA-Z]' OR shipment_id IS NULL" ).show()   # using Sql function

# 2. Age is not an integer
invalid_age_df = read_raw_df.filter((~col("age").rlike("^[0-9]+$"))).show()  # using Pyspark function
invalid_age_df = read_raw_df.where("age rlike '[a-zA-Z]' " ).show()          # using Sql function

# 3. Count rows having fewer columns than expected

from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType 

struttype1=StructType([StructField('ship_id', IntegerType(), True), StructField('first_name', StringType(), True), StructField('last_name', StringType(), True), StructField('age', ShortType(), True), StructField('role', StringType(), True),StructField("corruptedrows",StringType())])

df_1 = spark.read.schema(struttype1).csv(path="/Volumes/workspace/default/logistics_vol/logistics_data/logistics_source1",header=True,mode='permissive',columnNameOfCorruptRecord="corruptedrows")
#display(df_1)
fewer_df = df_1.where("size(split(corruptedrows, ',')) < 5")
display(fewer_df)
display(len(fewer_df.collect()))

# 4.Count rows having fewer columns than expected
more_df = df_1.where("size(split(corruptedrows, ',')) > 5")
display(more_df)
display(len(more_df.collect()))




In [0]:
#Create a Spark Session Object
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Enterprise Fleet/Logistics Analytics Pipeline").getOrCreate()


###**b. Active Data Munging** File: logistics_source1 and logistics_source2

#####1.Combining Data + Schema Merging (Structuring)
1. Read both files without enforcing schema
2. Align them into a single canonical schema: shipment_id,
first_name,
last_name,
age,
role,
hub_location,
vehicle_type,
data_source
3. Add data_source column with values as: system1, system2 in the respective dataframes

In [0]:
from pyspark.sql.functions import lit

# 1.Read both files without enforcing schema 
raw_data_df1 = spark.read.csv("/Volumes/workspace/default/logistics_vol/logistics_data/logistics_source1",header= True)
#display(raw_data_df1)
raw_data_df2 = spark.read.csv("/Volumes/workspace/default/logistics_vol/logistics_data/logistics_source2",header= True)
#display(raw_data_df2)

# 2.Add data_source column with values as: system1, system2 in the respective dataframes
raw_data_df1 = raw_data_df1.withColumn("data_source",lit("system1"))
raw_data_df2 = raw_data_df2.withColumn("data_source",lit("system2"))

# 3.Align them into a single canonical schema: shipment_id, first_name, last_name, age, role, hub_location, vehicle_type, data_source
schema_merged_df = raw_data_df1.unionByName(raw_data_df2,allowMissingColumns = True)
final_df = schema_merged_df.select("shipment_id","first_name","last_name","age","role","hub_location","vehicle_type","data_source")
display(final_df)


#####2. Cleansing, Scrubbing: 
Cleansing (removal of unwanted datasets)<br>
1. Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role<br>
2. Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name<br>
3. Join Readiness Rule - Drop records where the join key is null: shipment_id<br>

Scrubbing (convert raw to tidy)<br>
4. Age Defaulting Rule - Fill NULL values in the age column with: -1<br>
5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN<br>
6. Invalid Age Replacement - Replace the following values in age:
"ten" to -1
"" to -1<br>
7. Vehicle Type Normalization - Replace inconsistent vehicle types: 
truck to LMV
bike to TwoWheeler

In [0]:
# 1.Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role

cleansed_df1 = final_df.na.drop(how="any",subset=["shipment_id","role"])
#display(cleansed_df1)

# 2.Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name 
cleansed_df2 = cleansed_df1.na.drop(how="all",subset=["first_name","last_name"])
#display(cleansed_df2)

# 3.Join Readiness Rule - Drop records where the join key is null: shipment_id
#cleansed_df3 = cleansed_df2.filter(~col("shipment_id").rlike("[a-zA-Z]"))
cleansed_df3 = cleansed_df2.where("shipment_id not rlike '[a-zA-Z]' AND shipment_id IS NOT NULL")
#display(cleansed_df3)
cleansed_df3 = cleansed_df2.withColumn(
    "age",
    when(col("age").rlike("^[0-9]+$"),lit(None), col("age").cast("int"))
    .otherwise(None)
)

display(cleansed_df3)

# Scrubbing
# 4. Age Defaulting Rule - Fill NULL values in the age column with: -1
scrubbed_df1 = cleansed_df3.na.fill('-1',subset=["age"]) 
#display(scrubbed_df1)

# 5.Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN
scrubbed_df2 = scrubbed_df1.na.fill("UNKNOWN",subset=["vehicle_type"])
#display(scrubbed_df2)

# 6.Invalid Age Replacement - Replace the following values in age: "ten" to -1, "" to -1
find_replace = {"ten": "-1","": "-1" }
scrubbed_df3 = scrubbed_df2.na.replace(find_replace,subset=['age'])
#display(scrubbed_df3)

# 7.Vehicle Type Normalization - Replace inconsistent vehicle types: truck to LMV bike to TwoWheeler
find_replace_2 = {'Truck':'LMV','Bike':'TwoWheeler'}
scrubbed_df4 = scrubbed_df3.na.replace(find_replace_2,subset=["vehicle_type"])
display(scrubbed_df4)


####3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

Detail Dataframe creation <br>
1. Read Data from logistics_shipment_detail.json
2. As this data is a clean json data, it doesn't require any cleansing or scrubbing.

In [0]:
dfjson1 = spark.read.json("/Volumes/workspace/default/logistics_vol/logistics_data/logistics_shipment_detail_3000.json",multiLine=True)
dfjson1.printSchema()
display(dfjson1)

#str1="shipment_id int,order_id string,source_city string,destination_city string,shipment_status string,cargo_type string,vehicle_type string,payment_mode string,shipment_weight_kg double,shipment_cost double,shipment_date date"
#dfjson1=spark.read.schema(str1).json("/Volumes/workspace/default/logistics_vol/logistics_data/logistics_shipment_detail_3000.json",multiLine=True)
#display(dfjson1)

Standardizations:<br>

1. Add a column<br> 
Source File: DF of logistics_shipment_detail_3000.json<br>: domain as 'Logistics',  current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
2. Column Uniformity: 
role - Convert to lowercase<br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
vehicle_type - Convert values to UPPERCASE<br>
Source Files: DF of logistics_shipment_detail_3000.json
hub_location - Convert values to initcap case<br>
Source Files: DF of merged(logistics_source1 & logistics_source2)<br>
3. Format Standardization:<br>
Source Files: DF of logistics_shipment_detail_3000.json<br>
Convert shipment_date to yyyy-MM-dd<br>
Ensure shipment_cost has 2 decimal precision<br>
4. Data Type Standardization<br>
Standardizing column data types to fix schema drift and enable mathematical operations.<br>
Source File: DF of merged(logistics_source1 & logistics_source2) <br>
age: Cast String to Integer<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
shipment_weight_kg: Cast to Double<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
is_expedited: Cast to Boolean<br>
5. Naming Standardization <br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
Rename: first_name to staff_first_name<br>
Rename: last_name to staff_last_name<br>
Rename: hub_location to origin_hub_city<br>
6. Reordering columns logically in a better standard format:<br>
Source File: DF of Data from all 3 files<br>
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

In [0]:
#1.Add a column
#Source File: logistics_shipment_detail_3000.json: domain as 'Logistics' 
from pyspark.sql.functions import upper,lower,col,lit,initcap,lpad,to_date,when,current_timestamp
add_col_df = dfjson1.withColumn("domain",lit('Logistics'))
#display(add_col_df)

#2.Column Uniformity: role - Convert to lowercase
#Source File: logistics_source1 & logistics_source2
#vehicle_type - Convert values to UPPERCASE
#Source Files: logistics_shipment_detail_3000.json (and the merged master files) hub_location - Convert values to initcap case

col_uni_df = scrubbed_df4.withColumn("role",lower(col("role")))
#display(col_uni_df)
col_uni_df1= col_uni_df.withColumn('vehicle_type',upper(col('vehicle_type')))
#display(col_uni_df1)

master_df = scrubbed_df4.unionByName(add_col_df,allowMissingColumns= True)
display(master_df)

master_df = master_df.withColumn("hub_location",initcap("hub_location"))
#display(master_df)

#3.Format Standardization:
#Source Files: logistics_shipment_detail_3000.json Convert shipment_ref to string
#Pad to 10 characters with leading zeros
#Convert dispatch_date to yyyy-MM-dd
#Ensure delivery_cost has 2 decimal precision
df = master_df.withColumnRenamed("shipment_id","shipment_ref").withColumnRenamed("shipment_date","dispatch_date").withColumnRenamed("shipment_cost","delivery_cost")
format_fun_df = df.withColumn("shipment_id",lpad(col("shipment_id").cast("string"),10,"0"))
#display(df)

date_coversion_df = df.withColumn("dispatch_date",to_date(col("dispatch_date"),"yyyy/MM/dd"))
#display(date_coversion_df)

delivery_cost_df = df.withColumn("delivery_cost",col("delivery_cost").cast("double"))
#display(delivery_cost_df)

#4. Data Type Standardization
#Standardizing column data types to fix schema drift and enable mathematical operations.
#Source File: logistics_source1 & logistics_source2
#age: Cast String to Integer
#Source File: logistics_shipment_detail_3000.json
#shipment_weight_kg: Cast to Double
#Source File: logistics_shipment_detail_3000.json
#is_expedited: Cast to Boolean

dtype_df= scrubbed_df4.withColumn("shipment_id",col("shipment_id").cast("int")).withColumn("first_name",col("first_name").cast("string")).withColumn("last_name",col("last_name").cast("string")).withColumn("age",col("age").cast("int"))
dtype_df = dtype_df.withColumn("role",col("role").cast("string")).withColumn("vehicle_type",col("vehicle_type").cast("string")).withColumn("data_source",col("data_source").cast("string"))
#display(dtype_df)


dtype_df2 = dfjson1.withColumn("shipment_weight_kg",col("shipment_weight_kg").cast("double"))
#display(dtype_df2)

expedited_df = dfjson1.withColumn("is_expedited",when((col("shipment_status")=="IN_TRANSIT"),True).otherwise(False))
#display(expedited_df)

#5.Naming Standardization
#Source File: logistics_source1 & logistics_source2
#Rename: first_name to staff_first_name
#Rename: last_name to staff_last_name
#Rename: hub_location to origin_hub_city

naming_df = scrubbed_df4.withColumnsRenamed({'first_name':'staff_first_name','last_name':'staff_last_name','hub_location':'origin_hub_city'})
#display(naming_df)

#6.Reordering columns logically in a better standard format:
#Source File: All 3 files
#shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)



Deduplication:
1. Apply Record Level De-Duplication
2. Apply Column Level De-Duplication (Primary Key Enforcement)

In [0]:
display(master_df)
#display(master_df.count())

duplicate_df = master_df.dropDuplicates()
#display(duplicate_df.count())

col_duplicate_df = duplicate_df.dropDuplicates(["shipment_id"])
display(col_duplicate_df.count())


##2. Data Enrichment - Detailing of data
Makes your data rich and detailed <br>

###### Adding of Columns (Data Enrichment)
*Creating new derived attributes to enhance traceability and analytical capability.*

**1. Add Audit Timestamp (`load_dt`)**
Source File: logistics_source1 and logistics_source2<br>
* **Scenario:** We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
* **Action:** Add a column `load_dt` using the function `current_timestamp()`.

**2. Create Full Name (`full_name`)**
Source File: logistics_source1 and logistics_source2<br>
* **Scenario:** The reporting dashboard requires a single field for the driver's name instead of separate columns.
* **Action:** Create `full_name` by concatenating `first_name` and `last_name` with a space separator.
* **Result:** "Rajesh" + " " + "Kumar" -> **"Rajesh Kumar"**

**3. Define Route Segment (`route_segment`)**
Source File: logistics_shipment_detail_3000.json<br>
* **Scenario:** The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
* **Action:** Combine `source_city` and `destination_city` with a hyphen.
* **Result:** "Chennai" + "-" + "Pune" -> **"Chennai-Pune"**

**4. Generate Vehicle Identifier (`vehicle_identifier`)**
Source File: logistics_shipment_detail_3000.json<br>
* **Scenario:** We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
* **Action:** Combine `vehicle_type` and `shipment_id` to create a composite key.
* **Result:** "Truck" + "_" + "500001" -> **"Truck_500001"**

In [0]:
# 1. Add Audit Timestamp (load_dt) Source File: logistics_source1 and logistics_source2

'''Scenario: We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
Action: Add a column load_dt using the function current_timestamp().'''

audit_df = scrubbed_df4.withColumn("load_dt",current_timestamp())
display(audit_df)

#2. Create Full Name (full_name) Source File: logistics_source1 and logistics_source2

'''Scenario: The reporting dashboard requires a single field for the driver's name instead of separate columns.
Action: Create full_name by concatenating first_name and last_name with a space separator.
Result: "Rajesh" + " " + "Kumar" -> "Rajesh Kumar"'''
from pyspark.sql.functions import concat_ws
concat_df = audit_df.withColumn('full_name',concat_ws(" ",col("first_name"),col("last_name")))
display(concat_df)

#3. Define Route Segment (route_segment) Source File: logistics_shipment_detail_3000.json

'''Scenario: The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
Action: Combine source_city and destination_city with a hyphen.
Result: "Chennai" + "-" + "Pune" -> "Chennai-Pune"'''
from pyspark.sql.functions import concat_ws
concat_df1 = master_df.withColumn('source_destination',concat_ws("-",col("source_city"),col("destination_city")))
display(concat_df1)

#4. Generate Vehicle Identifier (vehicle_identifier) Source File: logistics_shipment_detail_3000.json

'''Scenario: We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
Action: Combine vehicle_type and shipment_id to create a composite key.
Result: "Truck" + "_" + "500001" -> "Truck_500001"'''
from pyspark.sql.functions import concat_ws
concat_df2 = master_df.withColumn('tracking_code',concat_ws("_",col("vehicle_type"),col("shipment_id")))
display(concat_df2)




###### Deriving of Columns (Time Intelligence)
*Extracting temporal features from dates to enable period-based analysis and reporting.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Derive Shipment Year (`shipment_year`)**
* **Scenario:** Management needs an annual performance report to compare growth year-over-year.
* **Action:** Extract the year component from `shipment_date`.
* **Result:** "2024-04-23" -> **2024**

**2. Derive Shipment Month (`shipment_month`)**
* **Scenario:** Analysts want to identify seasonal peaks (e.g., increased volume in December).
* **Action:** Extract the month component from `shipment_date`.
* **Result:** "2024-04-23" -> **4** (April)

**3. Flag Weekend Operations (`is_weekend`)**
* **Scenario:** The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
* **Action:** Flag as **'True'** if the `shipment_date` falls on a Saturday or Sunday.

In [0]:
#1. Derive Shipment Year (shipment_year)

'''Scenario: Management needs an annual performance report to compare growth year-over-year.
Action: Extract the year component from shipment_date.
Result: "2024-04-23" -> 2024'''

from pyspark.sql.functions import year
year_extract_df = master_df.select("*",year("shipment_date").alias("shipment_year"))
display(year_extract_df)

#2. Derive Shipment Month (shipment_month)

'''Scenario: Analysts want to identify seasonal peaks (e.g., increased volume in December).
Action: Extract the month component from shipment_date.
Result: "2024-04-23" -> 4 (April)'''

from pyspark.sql.functions import year,month,date_format
year_month_df = year_extract_df.select("*",date_format("shipment_date","MMMM").alias("shipment_month"))
display(year_month_df)

#3. Flag Weekend Operations (is_weekend)

'''Scenario: The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
Action: Flag as 'True' if the shipment_date falls on a Saturday or Sunday.
'''

from pyspark.sql.functions import dayofweek,when,col,when
flag_df = year_month_df.select("*",when(dayofweek(col("shipment_date")).isin([1,7]),True).otherwise(False).alias("flag"))
display(flag_df)

# OR

from pyspark.sql.functions import dayofweek,when,col
flag_df = year_month_df.withColumn("Flag",when(dayofweek(col('shipment_date')).isin([1,7]),True).otherwise(False))
display(flag_df)





###### Enrichment/Business Logics (Calculated Fields)
*Deriving new metrics and financial indicators using mathematical and date-based operations.*<br>
Source File: logistics_shipment_detail_3000.json<br>

**1. Calculate Unit Cost (`cost_per_kg`)**
* **Scenario:** The Finance team wants to analyze the efficiency of shipments by determining the cost incurred per unit of weight.
* **Action:** Divide `shipment_cost` by `shipment_weight_kg`.
* **Logic:** `shipment_cost / shipment_weight_kg`

**2. Track Shipment Age (`days_since_shipment`)**
* **Scenario:** The Operations team needs to monitor how long it has been since a shipment was dispatched to identify potential delays.
* **Action:** Calculate the difference in days between the `current_date` and the `shipment_date`.
* **Logic:** `datediff(current_date(), shipment_date)`

**3. Compute Tax Liability (`tax_amount`)**
* **Scenario:** For invoicing and compliance, we must calculate the Goods and Services Tax (GST) applicable to each shipment.
* **Action:** Calculate 18% GST on the total `shipment_cost`.
* **Logic:** `shipment_cost * 0.18`

###### Remove/Eliminate (drop, select, selectExpr)
*Excluding unnecessary or redundant columns to optimize storage and privacy.*<br>
Source File: logistics_source1 and logistics_source2<br>

**1. Remove Redundant Name Columns**
* **Scenario:** Since we have already created the `full_name` column in the Enrichment step, the individual name columns are now redundant and clutter the dataset.
* **Action:** Drop the `first_name` and `last_name` columns.
* **Logic:** `df.drop("first_name", "last_name")`

##### Splitting & Merging/Melting of Columns
*Reshaping columns to extract hidden values or combine fields for better analysis.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Splitting (Extraction)**
*Breaking one column into multiple to isolate key information.*
* **Split Order Code:**
  * **Action:** Split `order_id` ("ORD100000") into two new columns:
    * `order_prefix` ("ORD")
    * `order_sequence` ("100000")
* **Split Date:**
  * **Action:** Split `shipment_date` into three separate columns for partitioning:
    * `ship_year` (2024)
    * `ship_month` (4)
    * `ship_day` (23)

**2. Merging (Concatenation)**
*Combining multiple columns into a single unique identifier or description.*
* **Create Route ID:**
  * **Action:** Merge `source_city` ("Chennai") and `destination_city` ("Pune") to create a descriptive route key:
    * `route_lane` ("Chennai->Pune")

## 3. Data Customization & Processing - Application of Tailored Business Specific Rules

### **UDF1: Complex Incentive Calculation**
**Scenario:** The Logistics Head wants to calculate a "Performance Bonus" for drivers based on tenure and role complexity.

**Action:** Create a Python function `calculate_bonus(role, age)` and register it as a Spark UDF.

**Logic:**
* **IF** `Role` == 'Driver' **AND** `Age` > 50:
  * `Bonus` = 15% of Salary (Reward for Seniority)
* **IF** `Role` == 'Driver' **AND** `Age` < 30:
  * `Bonus` = 5% of Salary (Encouragement for Juniors)
* **ELSE**:
  * `Bonus` = 0

**Result:** A new derived column `projected_bonus` is generated for every row in the dataset.

---

### **UDF2: PII Masking (Privacy Compliance)**
**Scenario:** For the analytics dashboard, we must hide the full identity of the staff to comply with privacy laws (GDPR/DPDP), while keeping names recognizable for internal managers.

**Business Rule:** Show the first 2 letters, mask the middle characters with `****`, and show the last letter.

**Action:** Create a UDF `mask_identity(name)`.

**Example:**
* **Input:** `"Rajesh"`
* **Output:** `"Ra****h"`
<br>
**Note: Convert the above udf logic to inbult function based transformation to ensure the performance is improved.**

## 4. Data Core Curation & Processing (Pre-Wrangling)
*Applying business logic to focus, filter, and summarize data before final analysis.*

**1. Select (Projection)**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Scenario:** The Driver App team only needs location data, not sensitive HR info.
* **Action:** Select only `first_name`, `role`, and `hub_location`.

**2. Filter (Selection)**<br>
Source File: json<br>
* **Scenario:** We need a report on active operational problems.
* **Action:** Filter rows where `shipment_status` is **'DELAYED'** or **'RETURNED'**.
* **Scenario:** Insurance audit for senior staff.
* **Action:** Filter rows where `age > 50`.

**3. Derive Flags & Columns (Business Logic)**<br>
Source File: json<br>
* **Scenario:** Identify high-value shipments for security tracking.
* **Action:** Create flag `is_high_value` = **True** if `shipment_cost > 50,000`.
* **Scenario:** Flag weekend operations for overtime calculation.
* **Action:** Create flag `is_weekend` = **True** if day is Saturday or Sunday.

**4. Format (Standardization)**<br>
Source File: json<br>
* **Scenario:** Finance requires readable currency formats.
* **Action:** Format `shipment_cost` to string like **"₹30,695.80"**.
* **Scenario:** Standardize city names for reporting.
* **Action:** Format `source_city` to Uppercase (e.g., "chennai" → **"CHENNAI"**).

**5. Group & Aggregate (Summarization)**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Scenario:** Regional staffing analysis.
* **Action:** Group by `hub_location` and **Count** the number of staff.
* **Scenario:** Fleet capacity analysis.
* **Action:** Group by `vehicle_type` and **Sum** the `shipment_weight_kg`.

**6. Sorting (Ordering)**<br>
Source File: json<br>
* **Scenario:** Prioritize the most expensive shipments.
* **Action:** Sort by `shipment_cost` in **Descending** order.
* **Scenario:** Organize daily dispatch schedule.
* **Action:** Sort by `shipment_date` (Ascending) then `priority_flag` (Descending).

**7. Limit (Top-N Analysis)**<br>
Source File: json<br>
* **Scenario:** Dashboard snapshot of critical delays.
* **Action:** Filter for 'DELAYED', Sort by Cost, and **Limit to top 10** rows.

## 5. Data Wrangling - Transformation & Analytics
*Combining, modeling, and analyzing data to answer complex business questions.*

### **1. Joins**
Source Files:<br>
Left Side (staff_df):<br> logistics_source1 & logistics_source2<br>
Right Side (shipments_df):<br> logistics_shipment_detail_3000.json<br>
#### **1.1 Frequently Used Simple Joins (Inner, Left)**
* **Inner Join (Performance Analysis):**
  * **Scenario:** We only want to analyze *completed work*. Connect Staff to the Shipments they handled.
  * **Action:** Join `staff_df` and `shipments_df` on `shipment_id`.
  * **Result:** Returns only rows where a staff member is assigned to a valid shipment.
* **Left Join (Idle Resource check):**
  * **Scenario:** Find out which staff members are currently *idle* (not assigned to any shipment).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right) on `shipment_id`. Filter where `shipments_df.shipment_id` is NULL.

#### **1.2 Infrequent Simple Joins (Self, Right, Full, Cartesian)**
* **Self Join (Peer Finding):**
  * **Scenario:** Find all pairs of employees working in the same `hub_location`.
  * **Action:** Join `staff_df` to itself on `hub_location`, filtering where `staff_id_A != staff_id_B`.
* **Right Join (Orphan Data Check):**
  * **Scenario:** Identify shipments in the system that have *no valid driver* assigned (Data Integrity Issue).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right). Focus on NULLs on the left side.
* **Full Outer Join (Reconciliation):**
  * **Scenario:** A complete audit to find *both* idle drivers AND unassigned shipments in one view.
  * **Action:** Perform a Full Outer Join on `shipment_id`.
* **Cartesian/Cross Join (Capacity Planning):**
  * **Scenario:** Generate a schedule of *every possible* driver assignment to *every* pending shipment to run an optimization algorithm.
  * **Action:** Cross Join `drivers_df` and `pending_shipments_df`.

#### **1.3 Advanced Joins (Semi and Anti)**
* **Left Semi Join (Existence Check):**
  * **Scenario:** "Show me the details of Drivers who have *at least one* shipment." (Standard filtering).
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_semi")`.
  * **Benefit:** Performance optimization; it stops scanning the right table once a match is found.
* **Left Anti Join (Negation Check):**
  * **Scenario:** "Show me the details of Drivers who have *never* touched a shipment."
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_anti")`.

### **2. Lookup**<br>
Source File: logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Validation. Check if the `hub_location` in the staff file exists in the corporate `Master_City_List`.
* **Action:** Compare values against a reference list.

### **3. Lookup & Enrichment**<br>
Source File: logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Geo-Tagging.
* **Action:** Lookup `hub_location` ("Pune") in a Master Latitude/Longitude table and enrich the dataset by adding `lat` and `long` columns for map plotting.

### **4. Schema Modeling (Denormalization)**<br>
Source Files: All 3 Files (logistics_source1, logistics_source2, logistics_shipment_detail_3000.json)<br>
* **Scenario:** Creating a "Gold Layer" Table for PowerBI/Tableau.
* **Action:** Flatten the Star Schema. Join `Staff`, `Shipments`, and `Vehicle_Master` into one wide table (`wide_shipment_history`) so analysts don't have to perform joins during reporting.

### **5. Windowing (Ranking & Trends)**<br>
Source Files:<br>
logistics_source2: Provides hub_location (Partition Key).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Ordering Key)<br>
* **Scenario:** "Who are the Top 3 Drivers by Cost in *each* Hub?"
* **Action:**
  1. Partition by `hub_location`.
  2. Order by `total_shipment_cost` Descending.
  3. Apply `dense_rank()` and `row_number()
  4. Filter where `rank or row_number <= 3`.

### **6. Analytical Functions (Lead/Lag)**<br>
Source File: <br>
logistics_shipment_detail_3000.json<br>
* **Scenario:** Idle Time Analysis.
* **Action:** For each driver, calculate the days elapsed since their *previous* shipment.

### **7. Set Operations**<br>
Source Files: logistics_source1 and logistics_source2<br>
* **Union:** Combining `Source1` (Legacy) and `Source2` (Modern) into one dataset (Already done in Active Munging).
* **Intersect:** Identifying Staff IDs that appear in *both* Source 1 and Source 2 (Duplicate/Migration Check).
* **Except (Difference):** Identifying Staff IDs present in Source 2 but *missing* from Source 1 (New Hires).

### **8. Grouping & Aggregations (Advanced)**<br>
Source Files:<br>
logistics_source2: Provides hub_location and vehicle_type (Grouping Dimensions).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Aggregation Metric).<br>
* **Scenario:** The CFO wants a subtotal report at multiple levels:
  1. Total Cost by Hub.
  2. Total Cost by Hub AND Vehicle Type.
  3. Grand Total.
* **Action:** Use `cube("hub_location", "vehicle_type")` or `rollup()` to generate all these subtotals in a single query.

##6. Data Persistance (LOAD)-> Data Publishing & Consumption<br>

Store the inner joined, lookup and enrichment, Schema Modeling, windowing, analytical functions, set operations, grouping and aggregation data into the delta tables.

##7.Take the copy of the above notebook and try to write the equivalent SQL for which ever applicable.